<a href="https://colab.research.google.com/github/Kanka-max/tweets-transformers/blob/main/Zero_Shot_Pipeline_Condensed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## load transformer and initialize

!pip install transformers==3.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 884 kB 4.2 MB/s 
     |████████████████████████████████| 3.0 MB 47.1 MB/s 
     |████████████████████████████████| 880 kB 55.8 MB/s 
     |████████████████████████████████| 1.2 MB 65.4 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=84528a8c7a5e6359c895985204066bfebe4d9d776380b3e4896bd8bb52bba621
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.2 MB/s 
     |████████████████████████████████| 86 kB 2.8 MB/s 
     |████████████████████████████████| 362 kB 32.7 MB/s 
     |████████████████████████████████| 140 kB 33.8 MB/s 
     |████████████████████████████████| 212 kB 41.7 MB/s 
     |████████████████████████████████| 1.1 MB 41.9 MB/s 
     |████████████████████████████████| 596 kB 37.4 MB/s 
     |████████████████████████████████| 127 kB 51.0 MB/s 
     |████████████████████████████████| 144 kB 55.9 MB/s 
     |████████████████████████████████| 94 kB 3.9 MB/s 
     |████████████████████████████████| 271 kB 55.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
    

In [ ]:
from tqdm import tqdm
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
from transformers import pipeline

zero_classifier = pipeline("zero-shot-classification", model='joeddav/xlm-roberta-large-xnli')

Downloading:   0%|          | 0.00/734 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving en_200_examples.xlsx - en_200_examples.csv to en_200_examples.xlsx - en_200_examples.csv


In [ ]:
from google.colab import files

uploaded = files.upload()

Saving sw_200_examples.xlsx - sw_200_examples .csv to sw_200_examples.xlsx - sw_200_examples .csv


In [ ]:
import pandas as pd
df = pd.read_csv("sw_200_examples.xlsx - sw_200_examples .csv")
swa = df[["text", "label"]]

In [ ]:
import pandas as pd
df = pd.read_csv("en_200_examples.xlsx - en_200_examples.csv")
df.head()

,Gold Standard ID,Language,X,text,user_name,user_id,label
0,gs201,English,55378,@johnnjenga @BUANTAI004 @ShaffieWeru @mithika_...,Lisimba,3918809081,hateful
1,gs202,English,9717,How many Kikuyu spokesmen do we need in Mt. Ke...,"Peter Kariuki, M.Sc.",1390948818129203204,hateful
2,gs203,English,50071,@johnnjenga @ShaffieWeru @mithika_Linturi @SAB...,MkisiiWaNairobi,1352158178423828480,hateful
3,gs204,English,57783,Today’s “vijana” are the smartest but the lazi...,Rodney Musalia,839334020,hateful
4,gs205,English,46128,"In Azimio we only have two women, @gladyswanga...",Sir Breezy,1236327598214561793,hateful


In [ ]:

eng_df = df[["text", "label"]]

In [ ]:
from tqdm import tqdm


hateful        100
non-hateful    100
Name: label, dtype: int64

In [ ]:
%%time

## plots
## zero classifier
# loop through the text (tweets) col
scores = []
candidate_labels = ["hateful", "loving"]
hypothesis_template = "this sentence is {}."

for sequence in eng_df.text:
  # without multiclass
  score = zero_classifier(sequence, candidate_labels, hypothesis_template)
  scores.append(score)
df = pd.DataFrame.from_dict(scores)
df.reset_index(drop=True)

corpus_1 = df["labels"].apply(pd.Series)
corpus_2 = df["scores"].apply(pd.Series)

corpus_1.reset_index()
corpus_1.rename(columns = {0:'label_one'}, inplace = True)
corpus_1.rename(columns = {1:'label_two'}, inplace = True)
corpus_2.rename(columns = {0:'score_one'}, inplace = True)
corpus_2.rename(columns = {1:'score_two'}, inplace = True)
corpus = pd.concat([corpus_1, corpus_2], axis=1)

plot = corpus.groupby(["label_one","label_two"])["score_one","score_two"].mean().reset_index()
new_plot = plot[["label_one", "score_one", "label_two", "score_two"]]

fig = make_subplots(rows=2, cols=1, shared_yaxes=True, shared_xaxes=True, 
                    subplot_titles=("True Positive/ False Positive", "True Negative/False Negative"))

fig.add_trace(go.Bar(y=new_plot["label_one"], 
                     x=new_plot["score_one"],
                     name = "true positive/false positive pred",
                     orientation = "h",
                     marker=dict(color=[2, 5, 6], coloraxis="coloraxis")),
              1, 1)
fig.add_trace(go.Bar(y=new_plot["label_two"], 
                     x=new_plot["score_two"],
                     name = "true negative/false negative",
                     orientation = "h",
                     marker=dict(color=[2, 5, 6], coloraxis="coloraxis")),
              2, 1)


fig.update_layout(title = "Performance of RoBERTa-Large Model on English Labeled Tweets",
                  xaxis_title = "Model Score",
                  yaxis_title = "Class labels",
                  coloraxis=dict(colorscale='Bluered_r'), 
                  plot_bgcolor = "white",
                  barmode = "group",
                  showlegend=False)
fig.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


CPU times: user 4min 23s, sys: 2.45 s, total: 4min 25s
Wall time: 4min 29s


In [ ]:
new_plot

,label_one,score_one,label_two,score_two
0,hateful,0.662841,loving,0.337159
1,loving,0.661792,hateful,0.338208


In [ ]:
%%time
## swahili
## plots
## zero classifier
# loop through the text (tweets) col
scores = []
candidate_labels = ["hateful", "loving"]
hypothesis_template = "hii sentensi ni {}."

for sequence in swa.text:
  # without multiclass
  score = zero_classifier(sequence, candidate_labels, hypothesis_template)
  scores.append(score)
df = pd.DataFrame.from_dict(scores)
df.reset_index(drop=True)

corpus_1 = df["labels"].apply(pd.Series)
corpus_2 = df["scores"].apply(pd.Series)

corpus_1.reset_index()
corpus_1.rename(columns = {0:'label_one'}, inplace = True)
corpus_1.rename(columns = {1:'label_two'}, inplace = True)
corpus_2.rename(columns = {0:'score_one'}, inplace = True)
corpus_2.rename(columns = {1:'score_two'}, inplace = True)
corpus = pd.concat([corpus_1, corpus_2], axis=1)

plot = corpus.groupby(["label_one","label_two"])["score_one","score_two"].mean().reset_index()
new_plot = plot[["label_one", "score_one", "label_two", "score_two"]]

fig = make_subplots(rows=2, cols=1, shared_yaxes=True, shared_xaxes=True, 
                    subplot_titles=("True Positive/ False Positive", "True Negative/False Negative"))

fig.add_trace(go.Bar(y=new_plot["label_one"], 
                     x=new_plot["score_one"],
                     name = "true positive/false positive pred",
                     orientation = "h",
                     marker=dict(color=[3,4,5], coloraxis="coloraxis")),
              1, 1)
fig.add_trace(go.Bar(y=new_plot["label_two"], 
                     x=new_plot["score_two"],
                     name = "true negative/false negative",
                     orientation = "h",
                     marker=dict(color=[3,4,5], coloraxis="coloraxis")),
              2, 1)


fig.update_layout(title = "Performance of RoBERTa-Large Model on Swahili Labeled Tweets",
                  xaxis_title = "Model Score",
                  yaxis_title = "Class labels",
                  coloraxis=dict(colorscale='Bluered_r'), 
                  plot_bgcolor = "white",
                  barmode = "group",
                  showlegend=False)
fig.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



CPU times: user 4min 34s, sys: 5.05 s, total: 4min 39s
Wall time: 4min 43s


In [ ]:
# facebook model
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

Downloading:   0%|          | 0.00/908 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/bart-large-mnli were not used when initializing BartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing BartForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BartForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
%%time

## plots
## zero classifier
# loop through the text (tweets) col
scores = []
candidate_labels = ["hateful", "loving"]
hypothesis_template = "this sentence is {}."

for sequence in eng_df.text:
  # without multiclass
  score = zero_classifier(sequence, candidate_labels, hypothesis_template)
  scores.append(score)
df = pd.DataFrame.from_dict(scores)
df.reset_index(drop=True)

corpus_1 = df["labels"].apply(pd.Series)
corpus_2 = df["scores"].apply(pd.Series)

corpus_1.reset_index()
corpus_1.rename(columns = {0:'label_one'}, inplace = True)
corpus_1.rename(columns = {1:'label_two'}, inplace = True)
corpus_2.rename(columns = {0:'score_one'}, inplace = True)
corpus_2.rename(columns = {1:'score_two'}, inplace = True)
corpus = pd.concat([corpus_1, corpus_2], axis=1)

plot = corpus.groupby(["label_one","label_two"])["score_one","score_two"].mean().reset_index()
new_plot = plot[["label_one", "score_one", "label_two", "score_two"]]

fig = make_subplots(rows=2, cols=1, shared_yaxes=True, shared_xaxes=True, 
                    subplot_titles=("True Positive/ False Positive", "True Negative/False Negative"))

fig.add_trace(go.Bar(y=new_plot["label_one"], 
                     x=new_plot["score_one"],
                     name = "true positive/false positive pred",
                     orientation = "h",
                     marker=dict(color=[2, 5, 6], coloraxis="coloraxis")),
              1, 1)
fig.add_trace(go.Bar(y=new_plot["label_two"], 
                     x=new_plot["score_two"],
                     name = "true negative/false negative",
                     orientation = "h",
                     marker=dict(color=[2, 5, 6], coloraxis="coloraxis")),
              2, 1)


fig.update_layout(title = "Performance of Facebook/BART-Large Model on English Labeled Tweets",
                  xaxis_title = "Model Score",
                  yaxis_title = "Class labels",
                  coloraxis=dict(colorscale='Bluered_r'), 
                  plot_bgcolor = "white",
                  barmode = "group",
                  showlegend=False)
fig.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



CPU times: user 4min 15s, sys: 1.62 s, total: 4min 17s
Wall time: 4min 17s


In [ ]:
swa.label.value_counts()

hateful        100
non-hateful    100
Name: label, dtype: int64

In [ ]:
%%time
## swahili
## plots
## zero classifier
# loop through the text (tweets) col
scores = []
candidate_labels = ["hateful", "loving"]
hypothesis_template = "hii sentensi ni {}."

for sequence in swa.text:
  # without multiclass
  score = classifier(sequence, candidate_labels, hypothesis_template)
  scores.append(score)
df = pd.DataFrame.from_dict(scores)
df.reset_index(drop=True)

corpus_1 = df["labels"].apply(pd.Series)
corpus_2 = df["scores"].apply(pd.Series)

corpus_1.reset_index()
corpus_1.rename(columns = {0:'label_one'}, inplace = True)
corpus_1.rename(columns = {1:'label_two'}, inplace = True)
corpus_2.rename(columns = {0:'score_one'}, inplace = True)
corpus_2.rename(columns = {1:'score_two'}, inplace = True)
corpus = pd.concat([corpus_1, corpus_2], axis=1)

plot = corpus.groupby(["label_one","label_two"])["score_one","score_two"].mean().reset_index()
new_plot = plot[["label_one", "score_one", "label_two", "score_two"]]

fig = make_subplots(rows=2, cols=1, shared_yaxes=True, shared_xaxes=True, 
                    subplot_titles=("True Positive/ False Positive", "True Negative/False Negative"))

fig.add_trace(go.Bar(y=new_plot["label_one"], 
                     x=new_plot["score_one"],
                     name = "true positive/false positive pred",
                     orientation = "h",
                     marker=dict(color=[3,4,5], coloraxis="coloraxis")),
              1, 1)
fig.add_trace(go.Bar(y=new_plot["label_two"], 
                     x=new_plot["score_two"],
                     name = "true negative/false negative",
                     orientation = "h",
                     marker=dict(color=[3,4,5], coloraxis="coloraxis")),
              2, 1)


fig.update_layout(title = "Performance of Facebook/BART-Large Model on Swahili Labeled Tweets",
                  xaxis_title = "Model Score",
                  yaxis_title = "Class labels",
                  coloraxis=dict(colorscale='Bluered_r'), 
                  plot_bgcolor = "white",
                  barmode = "group",
                  showlegend=False)
fig.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



CPU times: user 6min 19s, sys: 2.99 s, total: 6min 22s
Wall time: 6min 37s


In [ ]:
#model pred and evaluation 
# metrics - precision, recall, accuracy, matthews correlation, 

# load dataset

In [ ]:
# load larger dataset ~ 15000
# use 1000 samples
from google.colab import files

uploaded = files.upload()

Saving Completed_Kiswahili_Hate_Speech_10.03.2022.xlsx - Completed_Kiswahili_Hate_Speech.csv to Completed_Kiswahili_Hate_Speech_10.03.2022.xlsx - Completed_Kiswahili_Hate_Speech (1).csv


In [ ]:
df = pd.read_csv("Completed_Kiswahili_Hate_Speech_10.03.2022.xlsx - Completed_Kiswahili_Hate_Speech.csv")
temp_df = df.iloc[0:1000]

In [ ]:
# get unique values
# get sample size for each class
temp_df.does_the_text_contain_hate_speech.value_counts()


non-hateful    973
hateful         27
Name: does_the_text_contain_hate_speech, dtype: int64

In [ ]:
%%time

category_map = {"hateful": "hate", "non-hateful": "loving"}

candidate_labels = list(category_map.values())
 #candidate labels are basically the classes that the classifier will predict
predictedCategories = []
trueCategories = []
for i in tqdm(range(1000)):
    text = temp_df.iloc[i,]['text']
    cat = temp_df.iloc[i,]['does_the_text_contain_hate_speech']
    cat = cat.split()
    res = classifier(text, candidate_labels)#setting multi-class as True
    labels = res['labels'] 
    scores = res['scores'] #extracting the scores associated with the labels
    res_dict = {label : score for label,score in zip(labels, scores)}
    sorted_dict = dict(sorted(res_dict.items(), key=lambda x:x[1],reverse = True)) #sorting the dictionary of labels in descending order based on their score
    categories  = []
    for i, (k,v) in enumerate(sorted_dict.items()):
        if(i > 2): #storing only the best 3 predictions
            break
        else:
            categories.append(k)
    predictedCategories.append(categories)
    trueCats = [category_map[x] for x in cat]
    trueCategories.append(trueCats)

100%|██████████| 1000/1000 [26:06<00:00,  1.57s/it]

CPU times: user 25min 55s, sys: 15.6 s, total: 26min 11s
Wall time: 26min 6s


In [ ]:
for y_true, y_pred in zip(trueCategories[:2], predictedCategories[:2]):
    print(f'True Categories {y_true}')
    print(f'Predicted Categories {y_pred}')
    print('#'*50)

True Categories ['hate']
Predicted Categories ['hate', 'loving']
##################################################
True Categories ['hate']
Predicted Categories ['hate', 'loving']
##################################################


In [ ]:
category_idx = {cat : i for i,cat in enumerate(category_map.values())} 
y_trueEncoded = []
y_predEncoded = []
for y_true, y_pred in zip(trueCategories, predictedCategories):
    encTrue = [0] * len(category_map)
    for cat in y_true:
        idx = category_idx[cat]
        encTrue[idx] = 1
    y_trueEncoded.append(encTrue)
    encPred = [0] * len(category_map)
    for cat in y_pred:
        idx = category_idx[cat]
        encPred[idx] = 1
    y_predEncoded.append(encPred)

In [ ]:
import evaluate

In [ ]:
precision_metric = evaluate.load("precision")
results = precision_metric.compute(references=[0, 1,1,0], predictions=[1,1,1,1])
print(results)

{'precision': 0.5}


In [ ]:
f1_metric = evaluate.load("f1")
results = f1_metric.compute(references=[0, 1,1,0], predictions=[1, 1, 1, 1])
print(results)

{'f1': 0.6666666666666666}


In [ ]:
recall_metric = evaluate.load('recall')
results = recall_metric.compute(references=[0, 1,1,0], predictions=[1, 1, 1, 1])
print(results)

{'recall': 1.0}


In [ ]:
## matthews_metric

matthews_metric = evaluate.load("matthews_correlation")
results = matthews_metric.compute(references=[1,1], predictions=[1,0])
print(results)


{'matthews_correlation': 0.0}


In [ ]:
from sklearn.metrics import hamming_loss
print(f'The hamming loss is {hamming_loss(y_trueEncoded,y_predEncoded):.4f}')

The hamming loss is 0.5000


**Metric Card for Matthews Correlation Coefficient**

*Metric Description*

The Matthews correlation coefficient is used in machine learning as a measure of the quality of binary and multiclass classifications. It takes into account true and false positives and negatives and is generally regarded as a balanced measure which can be used even if the classes are of very different sizes. The MCC is in essence a correlation coefficient value between -1 and +1. A coefficient of +1 represents a perfect prediction, 0 an average random prediction and -1 an inverse prediction. The statistic is also known as the phi coefficient

In [ ]:
#metric requires a list of predictions and a list of references:
#predictions (list of ints): Predicted class labels.
#references (list of ints): Ground truth labels

In [ ]:
matthews_metric = evaluate.load("matthews_correlation")
results = matthews_metric.compute(references=[0, 1], predictions=[0, 1])
print(results)
{'matthews_correlation': 1.0}

In [ ]:
%%time

category_map = {"hateful": "hate", "non-hateful": "loving"}

candidate_labels = list(category_map.values())
 #candidate labels are basically the classes that the classifier will predict
predictedCategories = []
trueCategories = []
for i in tqdm(range(1000)):
    text = temp_df.iloc[i,]['text']
    cat = temp_df.iloc[i,]['does_the_text_contain_hate_speech']
    cat = cat.split()
    res = zero_classifier(text, candidate_labels)#setting multi-class as True
    labels = res['labels'] 
    scores = res['scores'] #extracting the scores associated with the labels
    res_dict = {label : score for label,score in zip(labels, scores)}
    sorted_dict = dict(sorted(res_dict.items(), key=lambda x:x[1],reverse = True)) #sorting the dictionary of labels in descending order based on their score
    categories  = []
    for i, (k,v) in enumerate(sorted_dict.items()):
        if(i > 2): #storing only the best 3 predictions
            break
        else:
            categories.append(k)
    predictedCategories.append(categories)
    trueCats = [category_map[x] for x in cat]
    trueCategories.append(trueCats)

100%|██████████| 1000/1000 [23:47<00:00,  1.43s/it]

CPU times: user 23min 42s, sys: 10.2 s, total: 23min 52s
Wall time: 23min 47s
